In [5]:
!wget https://datahack-prod.s3.amazonaws.com/test_file/test_8iecVfC.zip
!wget https://datahack-prod.s3.amazonaws.com/train_file/train_tGmol3O.zip
!wget https://datahack-prod.s3.amazonaws.com/sample_submission/sample_submission_UVKGLZE.csv

--2020-08-23 19:05:59--  https://datahack-prod.s3.amazonaws.com/test_file/test_8iecVfC.zip
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.62.44
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.62.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3537322 (3.4M) [application/zip]
Saving to: ‘test_8iecVfC.zip’

test_8iecVfC.zip    100%[===================>]   3.37M  1.79MB/s    in 1.9s    

2020-08-23 19:06:02 (1.79 MB/s) - ‘test_8iecVfC.zip’ saved [3537322/3537322]

--2020-08-23 19:06:04--  https://datahack-prod.s3.amazonaws.com/train_file/train_tGmol3O.zip
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.66.80
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.66.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8286441 (7.9M) [application/zip]
Saving to: ‘train_tGmol3O.zip’

train_tGmol3O.zip 

In [7]:
!unzip train_tGmol3O.zip
!unzip test_8iecVfC.zip

Archive:  train_tGmol3O.zip
  inflating: train.csv               
Archive:  test_8iecVfC.zip
  inflating: test.csv                


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission_UVKGLZE.csv')

In [22]:
train.head(5)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [23]:
test.head(5)

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [24]:
sub.head(5)

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [25]:
train.isnull().sum(),test.isnull().sum(),train.shape,test.shape,train.dtypes

(ID                      0
 TITLE                   0
 ABSTRACT                0
 Computer Science        0
 Physics                 0
 Mathematics             0
 Statistics              0
 Quantitative Biology    0
 Quantitative Finance    0
 dtype: int64, ID          0
 TITLE       0
 ABSTRACT    0
 dtype: int64, (20972, 9), (8989, 3), ID                       int64
 TITLE                   object
 ABSTRACT                object
 Computer Science         int64
 Physics                  int64
 Mathematics              int64
 Statistics               int64
 Quantitative Biology     int64
 Quantitative Finance     int64
 dtype: object)

In [26]:
#for F1
train['Department']=train['ABSTRACT']+' '+train['TITLE']
test['Department']=test['ABSTRACT']+' '+test['TITLE']
del train['ID']
del test['ID']
del train['ABSTRACT']
del test['ABSTRACT']
del train['TITLE']
del test['TITLE']

In [27]:
train['labels'] = list(zip(train['Computer Science'].tolist(), train['Physics'].tolist(), train['Mathematics'].tolist(), train['Statistics'].tolist(),  train['Quantitative Biology'].tolist(), train['Quantitative Finance'].tolist()))
train['Department'] = train['Department'].apply(lambda x: x.replace('\n', ' '))

In [28]:
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

In [29]:
from simpletransformers.classification import MultiLabelClassificationModel
model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6, args={'train_batch_size':32, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 6, 'max_seq_length': 256})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [ ]:
model.train_model(train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:282: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [ ]:
to_predict = test.text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)

In [ ]:
label_cols = ['Computer Science', 'Physics' , 'Mathematics' , 'Statistics' , 'Quantitative Biology' , 'Quantitative Finance']

In [ ]:
submission = pd.DataFrame({'ID': test_id})
submission = pd.concat([submission, pd.DataFrame(outputs, columns = label_cols)], axis=1)

submission.head()

In [ ]:
submission.to_csv('av.csv', index=False)